In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","notifica")

if not isdir(output):
    makedirs(output)

In [2]:
timerAll = Timer()

classificacao_final = ['0','1','2','3','5']

timer = Timer()
notifica = Notifica()
# notifica.read(download_metabase(filename='null.csv',where=f"classificacao_final IS NULL"))
# notifica.read(join('input','queries','null.csv'))
# for cf in classificacao_final:
    # notifica.read(download_metabase(filename=f"{cf}.csv",where=f"classificacao_final = {cf}"), append=True)
    # notifica.read(join('input','queries',f"{cf}.csv"), append=True)


# notifica.normalize()
# notifica.save()
notifica.load()
print(timer.stop())
notifica.shape()

6.029874563217163


(1982760, 885087, 8816, 1086961, 1896)

In [3]:
timer.reset()
casos_confirmados = CasosConfirmados()
casos_confirmados.update()
print(timer.stop())
casos_confirmados.shape()

Casos confirmados excluidos: 4206
Obitos confirmados excluidos: 0
254.13665556907654


(488448, 8874)

In [4]:
casosn = notifica.get_casos()
casosc = casos_confirmados.get_casos()
casosc = casosc.loc[casosc['excluir']!='SIM']
casosn.groupby('classificacao_final')[['id']].count()

,id
classificacao_final,
CASO CONFIRMADO,463143
CASO DESCARTADO,656766
CASO SUSPEITO,855138
IGNORADO,692
SINDROME GRIPAL NAO ESPECIFICADA,7021


In [5]:
casos_duplicados = casosn.loc[(casosn['cpf'].notnull()) & (casosn.duplicated('cpf',keep=False))].sort_values('cpf')
print(f"Total de pacientes com mais de uma ocorrencia: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('cpf'))} pacientes pelo CPF que estavam com mais de uma ocorrencia")

all_casos_duplicados = set()
all_duplicados_mantidos = set()
manter = []

for hash, group in casos_duplicados.groupby('cpf'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_cpf.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia: 407252
165712 pacientes pelo CPF que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,11425
CASO DESCARTADO,93971
CASO SUSPEITO,134865
IGNORADO,45
SINDROME GRIPAL NAO ESPECIFICADA,1234


In [6]:
casosn = casosn.drop(index=casos_duplicados)

In [7]:
casos_duplicados = casosn.loc[(casosn['nome_mae'].notnull()) & (casosn.duplicated('hash_mae',keep=False))].sort_values('nome_mae')
# casos_duplicados_nome_mae.loc[casos_duplicados_nome_mae['nome_mae']=='0']

print(f"Total de pacientes com mais de uma ocorrencia pelo nome+nome_mae: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('nome_mae'))} pacientes pelo nome+nome_mae que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('nome_mae'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_mae.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo nome+nome_mae: 67331
28656 pacientes pelo nome+nome_mae que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,2436
CASO DESCARTADO,13851
CASO SUSPEITO,22199
IGNORADO,5
SINDROME GRIPAL NAO ESPECIFICADA,184


In [8]:
casosn = casosn.drop(index=casos_duplicados)

In [9]:
casos_duplicados = casosn.loc[(casosn['hash_nasc'].notnull()) & (casosn.duplicated('hash_nasc',keep=False))].sort_values('data_nascimento')
# casos_duplicados_nome_mae.loc[casos_duplicados_nome_mae['nome_mae']=='0']

print(f"Total de pacientes com mais de uma ocorrencia pelo data_nascimento: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_nasc'))} pacientes pelo data_nascimento que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_nasc'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_data_nascimento.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo data_nascimento: 17804
8801 pacientes pelo data_nascimento que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,796
CASO DESCARTADO,2710
CASO SUSPEITO,5458
IGNORADO,7
SINDROME GRIPAL NAO ESPECIFICADA,32


In [10]:
casosn = casosn.drop(index=casos_duplicados)

In [11]:
casos_duplicados = casosn.loc[(casosn['hash_idade_resid'].notnull()) & (casosn.duplicated('hash_idade_resid',keep=False))].sort_values('paciente')

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_idade_resid: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_idade_resid'))} pacientes pelo hash_idade_resid que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_idade_resid'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_resid.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo hash_idade_resid: 2204
1081 pacientes pelo hash_idade_resid que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,58
CASO DESCARTADO,219
CASO SUSPEITO,844
SINDROME GRIPAL NAO ESPECIFICADA,2


In [12]:
casosn = casosn.drop(index=casos_duplicados)

In [13]:
casos_duplicados = casosn.loc[(casosn['hash_idade_atend'].notnull()) & (casosn.duplicated('hash_idade_atend',keep=False))].sort_values('paciente')
# casos_duplicados_nome_mae.loc[casos_duplicados_nome_mae['nome_mae']=='0']

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_idade_atend: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_idade_atend'))} pacientes pelo hash_idade_atend que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_idade_atend'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_atend.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo hash_idade_atend: 308
154 pacientes pelo hash_idade_atend que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,13
CASO DESCARTADO,31
CASO SUSPEITO,108
IGNORADO,2


In [14]:
casosn = casosn.drop(index=casos_duplicados)

In [15]:
print(f"Das {notifica.shape()[0]} notificações baixadas, {notifica.shape()[0] - casosn.shape[0]} ({len(all_casos_duplicados)}) foram excluidas por serem do mesmo paciente, sendo mantida a que melhor pontuou nos critérios. Sendo assim, o novo total de notificações existente é de {casosn.shape[0]}")
allcasosn = notifica.get_casos()
allcasosn.loc[all_casos_duplicados,'id'].to_csv(join(output,'casos_duplicados_removidos.csv'), index=False)
allcasosn.loc[all_duplicados_mantidos,'id'].to_csv(join(output,'casos_duplicados_mantidos.csv'), index=False)

Das 1982760 notificações baixadas, 290495 (290495) foram excluidas por serem do mesmo paciente, sendo mantida a que melhor pontuou nos critérios. Sendo assim, o novo total de notificações existente é de 1692265


In [16]:
idx_casos_confirmados = casosc[casosc['hash'].isin(casosn['hash_idade_resid'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less'].isin(casosn['hash_idade_resid'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more'].isin(casosn['hash_idade_resid'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casosn['hash_idade_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less_atend'].isin(casosn['hash_idade_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more_atend'].isin(casosn['hash_idade_atend'])].index.tolist()
idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
# casos_confirmados_nao_notifica.to_excel(join(output,'casos_confirmados_nao_notifica.xlsx'),index=False)
casos_confirmados_nao_notifica.shape

(27738, 22)

In [17]:
idx_casos_notifica = casosn[casosn['hash_idade_resid'].isin(casosc['hash'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_idade_resid'].isin(casosc['hash_less'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_idade_resid'].isin(casosc['hash_more'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_idade_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_idade_atend'].isin(casosc['hash_less_atend'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_idade_atend'].isin(casosc['hash_more_atend'])].index.tolist()
idx_casos_notifica = set(idx_casos_notifica)
casos_confirmados_notifica = casosn.loc[ idx_casos_notifica ].sort_values('id')
# casos_confirmados_notifica.to_excel(join(output,'casos_confirmados_notifica.xlsx'),index=False)
print(timerAll.stop())
casos_confirmados_notifica.shape

484.3765459060669


(459020, 55)

In [18]:
notifica_nao_casos_confirmados = casosn.loc[set(casosn.index.tolist()) - set(idx_casos_notifica)]

In [19]:
notifica_nao_casos_confirmados = notifica_nao_casos_confirmados.loc[notifica_nao_casos_confirmados['cod_classificacao_final'] == 2]
notifica_nao_casos_confirmados = notifica_nao_casos_confirmados.loc[notifica_nao_casos_confirmados['cod_status_notificacao'].isin([1,2])]

notifica_nao_casos_confirmados['month'] = notifica_nao_casos_confirmados.apply(lambda x: x['data_notificacao'].month, axis=1)
nnccm = notifica_nao_casos_confirmados.groupby(by='month')[['id']].count()
nnccm.index = [ meses[int(x)-1] for x in nnccm.index] 
nnccm

,id
jan,304
mar,8
abril,25
mai,46
jun,139
jul,290
ago,207
set,69
out,26
nov,143


In [20]:
# notifica_nao_casos_confirmados.to_excel(join(output,'notifica_nao_casos_confirmados.xlsx'), index=False)
notifica_nao_casos_confirmados.shape

(1563, 56)